# Chapter 9

In [1]:
#Test code
#import tensorflow as tf

#x=tf.Variable(3,name="x")
#y=tf.Variable(4,name="y")
#f=x*x*y+y+2

In [2]:
from sklearn.datasets import make_moons

m=100
X,y = make_moons(m, noise=0.1, random_state=42)

In [3]:
import numpy as np

X_moons_with_bias = np.c_[np.ones((m, 1)), X]
#X_moons_with_bias[:5]

In [4]:
y_moons_column_vector = y.reshape(-1, 1)
#y_moons_column_vector

In [5]:
#Split into test and training data
test_ratio = 0.2
test_size = int(m * test_ratio)
make_moons_plus_bias = np.c_[np.ones((m, 1)), X[:,0]]
X_train = X_moons_with_bias[:-test_size]
X_test = X_moons_with_bias[-test_size:]
y_train = y_moons_column_vector[:-test_size]
y_test = y_moons_column_vector[-test_size:]

In [6]:
def fetch_batch(epoch, batch_index, batch_size):
#    np.random.seed(epoch * n_batches + batch_index)
    indices = np.random.randint(0, len(X_train), size=batch_size)
    X_batch = X_train[indices]
    y_batch = y_train[indices]
    
    return X_batch, y_batch

In [7]:
import tensorflow as tf

# to make this notebook's output stable across runs
def reset_graph(seed=42):
    tf.reset_default_graph()
    tf.set_random_seed(seed)
    np.random.seed(seed)

reset_graph()

In [8]:
n_inputs = 2

X=tf.placeholder(tf.float32, shape=(None, n_inputs+1), name="X")
y=tf.placeholder(tf.float32, shape=(None, 1), name="y")

theta = tf.Variable(tf.random_uniform([n_inputs + 1, 1], -1.0, 1.0, seed=42), name="theta")
logits = tf.matmul(X, theta, name="logits")
y_prob = 1/(1+tf.exp(-logits))

epsilon = 1e-7
loss = -tf.reduce_mean(y*tf.log(y_prob + epsilon)+(1-y)*tf.log(1-y_prob + epsilon))

learning_rate=0.01
optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate)
training_op = optimizer.minimize(loss)

In [9]:
init = tf.global_variables_initializer()
#saver = tf.train.Saver()

In [10]:
batch_size = 50
n_batches = int(np.ceil(m/batch_size))
n_epochs = 1000

with tf.Session() as sess:
    sess.run(init)

    for epoch in range(n_epochs):
        for batch_index in range(n_batches):
            X_batch, y_batch = fetch_batch(X_train, y_train, batch_size)
            sess.run(training_op, feed_dict={X: X_batch, y: y_batch})
#            save_path = saver.save(sess, "C:/Users/Prathusha/Documents/GitHub/Special_Project/9.ckpt")
        loss_val = loss.eval({X: X_test, y: y_test})
        if epoch % 100 == 0:
            print("Epoch:", epoch, "\tLoss:", loss_val)

    y_prob_val = y_prob.eval(feed_dict={X: X_test, y: y_test})

#    save_path = saver.save(sess, "Documents/GitHub/Special_Project/9_final.ckpt")

Epoch: 0 	Loss: 0.85739815
Epoch: 100 	Loss: 0.6119874
Epoch: 200 	Loss: 0.5220956
Epoch: 300 	Loss: 0.4782013
Epoch: 400 	Loss: 0.45339537
Epoch: 500 	Loss: 0.43802196
Epoch: 600 	Loss: 0.42696577
Epoch: 700 	Loss: 0.41886368
Epoch: 800 	Loss: 0.4126358
Epoch: 900 	Loss: 0.40772143


In [11]:
y_prob_val[:5]

array([[0.68571544],
       [0.8930229 ],
       [0.09778903],
       [0.11237805],
       [0.66596496]], dtype=float32)

In [12]:
y_pred = (y_prob_val >= 0.5)
y_pred[:5]

array([[ True],
       [ True],
       [False],
       [False],
       [ True]])